In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/stream.db',True)
dbutils.fs.rm('dbfs:/FileStore/streaming',True)

Out[1]: True

In [0]:
%sql
DROP DATABASE IF EXISTS stream CASCADE;
CREATE DATABASE IF NOT EXISTS stream


## AutoLoader

In [0]:
source_dir = 'dbfs:/FileStore/streaming/'

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('header','true')\
        .load(source_dir)

In [0]:
dbutils.fs.ls(f'{source_dir}/schemaInfer')

Out[13]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/', name='_schemas/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls('dbfs:/FileStore/streaming/schemaInfer/_schemas/')

Out[14]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/0', name='0', size=274, modificationTime=1701879153000)]

In [0]:
%sql

SELECT *
FROM JSON.`dbfs:/FileStore/streaming/schemaInfer/_schemas/0`

_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Country"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Citizens"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"


### SchemaHints

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('cloudFiles.schemaHints',"Citizens LONG")\
        .option('header','true')\
        .load(source_dir)

In [0]:
display(df)

Country,Citizens,_rescued_data
India,10,null
USA,5,null
China,10,null
India,10,null
Canada,40,null
Brazil,10,null


In [0]:
%sql

SELECT *
FROM JSON.`dbfs:/FileStore/streaming/schemaInfer/_schemas/0`

_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Country"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Citizens"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"



## Schema Evolution

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option('cloudFiles.schemaEvolutionMode','rescue')\
        .option('rescuedDataColumn','_rescued_data')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('cloudFiles.schemaHints',"Citizens LONG")\
        .option('header','true')\
        .load(source_dir)

In [0]:
display(df)

Country,Citizens,_rescued_data
India,10,"{""Month"":""November"",""Year"":""2023"",""_file_path"":""dbfs:/FileStore/streaming/Countries_newcolumn1.csv""}"
USA,10,"{""Month"":""November"",""Year"":""2023"",""_file_path"":""dbfs:/FileStore/streaming/Countries_newcolumn1.csv""}"
China,20,"{""Month"":""November"",""Year"":""2023"",""_file_path"":""dbfs:/FileStore/streaming/Countries_newcolumn1.csv""}"
Brazil,10,"{""Month"":""November"",""Year"":""2023"",""_file_path"":""dbfs:/FileStore/streaming/Countries_newcolumn1.csv""}"
India,10,null
USA,5,null
China,10,null
India,10,null
Canada,40,null
Brazil,10,null


In [0]:
%sql

SELECT *
FROM JSON.`dbfs:/FileStore/streaming/schemaInfer/_schemas/0`

_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Country"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Citizens"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"


In [0]:
dbutils.fs.ls('dbfs:/FileStore/streaming/schemaInfer/_schemas/')

Out[22]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/0', name='0', size=274, modificationTime=1701879153000)]


### 02 - addNewColumns - Default

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('cloudFiles.schemaHints',"Citizens LONG")\
        .option('header','true')\
        .load(source_dir)

In [0]:
display(df)

Country,Citizens,Year,Month,_rescued_data
India,10,2023,November,null
USA,10,2023,November,null
China,20,2023,November,null
Brazil,10,2023,November,null
India,10,null,null,null
USA,5,null,null,null
China,10,null,null,null
India,10,null,null,null
Canada,40,null,null,null
Brazil,10,null,null,null


In [0]:
dbutils.fs.ls('dbfs:/FileStore/streaming/schemaInfer/_schemas/')

Out[25]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/0', name='0', size=274, modificationTime=1701879153000),
 FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/1', name='1', size=424, modificationTime=1701883002000)]

In [0]:
%sql

SELECT *
FROM JSON.`dbfs:/FileStore/streaming/schemaInfer/_schemas/1`

_corrupt_record,dataSchemaJson,partitionSchemaJson
v1,null,null
null,"{""type"":""struct"",""fields"":[{""name"":""Country"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Citizens"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Month"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[]}"



### 03- failOnNewColumns

In [0]:
dbutils.fs.ls('dbfs:/FileStore/streaming/schemaInfer/_schemas/')

Out[33]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/0', name='0', size=274, modificationTime=1701879153000)]

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option('cloudFiles.schemaEvolutionMode','failOnNewColumns')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('cloudFiles.schemaHints',"Citizens LONG")\
        .option('header','true')\
        .load(source_dir)

In [0]:
display(df)

In [0]:
dbutils.fs.ls('dbfs:/FileStore/streaming/schemaInfer/_schemas/')

Out[36]: [FileInfo(path='dbfs:/FileStore/streaming/schemaInfer/_schemas/0', name='0', size=274, modificationTime=1701879153000)]


### None

In [0]:
df = spark.readStream\
        .format('cloudFiles')\
        .option("cloudFiles.format","csv")\
        .option("cloudFiles.schemaLocation",f'{source_dir}/schemaInfer')\
        .option('cloudFiles.schemaEvolutionMode','none')\
        .option("cloudFiles.inferColumnTypes","true")\
        .option('cloudFiles.schemaHints',"Citizens LONG")\
        .option('header','true')\
        .load(source_dir)

In [0]:
display(df)

Country,Citizens
India,10
USA,10
China,20
Brazil,10
India,10
USA,10
China,20
Brazil,10
India,10
USA,5
